<a href="https://colab.research.google.com/github/kwakseoyeon/1test/blob/master/getting_started_with_openai_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAI API(챗 완성) 사용해보기

In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import json

# 개발자가 연동을 하고 싶은 함수를 정의한다.
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit = "celsius"):

    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }

    return json.dumps(weather_info)

In [ ]:
ret = get_current_weather('서울')

In [ ]:
print(ret)

{"location": "\uc11c\uc6b8", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [ ]:
def get_gps():
    return '서울'

In [ ]:
# 함수 호출을 위한 dict

available_functions = {
    "get_current_weather": get_current_weather,
    "get_location_from_gps" : get_gps, # 일부러 이름을 다르게 함 (이유: 이해를 돕기 위해)
}

In [ ]:
ret = available_functions["get_current_weather"]('서울')

print(ret)

{"location": "\uc11c\uc6b8", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [ ]:
ret = available_functions["get_location_from_gps"]()

print(ret)

서울


In [ ]:
def run_conversation(intput_text):
    # Step 1: send the conversation and available functions to GPT
    messages = [
        {"role": "user", "content": intput_text}
    ]

    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )

    print(response)

    response_message = response["choices"][0]["message"]

    # 함수 특정에 성공하면 "function_call" 키가 존재합니다.
    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"): # 함수 특정에 성공했다면,
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 인자를 얻어온다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # 함수를 호출해서 그 결과를 얻는다.
        function_response = available_functions[function_name](
            location = function_args.get("location"),
            unit = function_args.get("unit"),
        )

        # response_message : function_call, name, arguments
        # 함수 이름과 인자 정보를 담은 메시지를 추가한다.
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수 호출 결과를 메시지에 추가한다.
        messages.append(
            {
                "role": "function", # 역할 : 함수 호출 결과는 역할을 function으로 설정한다.
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = messages, # 사용자 요청사항, 함수 이름 및 인자, 함수 호출 결과
        )  # get a new response from GPT where it can see the function response

        return second_response
    else:
        return response

In [ ]:
response = run_conversation("What's the weather in Seoul?")

print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-97fsRzxNjyUYRYmrQSL1Vp9GmatYW",
  "object": "chat.completion",
  "created": 1711616299,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 16,
    "total_tokens": 97
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': "What's the weather in Seoul?"}, <OpenAIObject at 0x7d1da8205da0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\":

In [ ]:
response = run_conversation("What's the weather?")
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-97fsVk0p9J7n6k4gfgpmckNdf2txa",
  "object": "chat.completion",
  "created": 1711616303,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "In order to provide you with information about the current weather, could you please provide me with the location (city and state)?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 26,
    "total_tokens": 105
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": "In order to provide you with information about the current weather, could you please provide me with the location (city and state)?"
}
In order to provide you with information about the current weather, could you please provide me with the location (city and state)?


In [ ]:
def run_conversation(user_input_text):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": user_input_text}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_location_from_gps",
            "description": "Get the current location",
            "parameters": {
                "type": "object",
                "properties": {
                    "unit": {"type": "string"},
                },
                "required": [],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-0613",
        messages = messages,
        functions = functions,
        function_call = "auto",  # auto is default, but we'll be explicit
    )

    print(response)

    response_message = response["choices"][0]["message"]

    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 함수 인자를 얻는다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # 함수 호출
        # 어떤 함수가 호출되는 지 모르기 때문에, 즉 인자를 특정할 수 없다.
        function_response = available_functions[function_name](**function_args)

        # [중요!!] 함수 이름과 인자 정보 추가
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수 호출 결과 추가 (role : function)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response
    else:
        return response_message

In [ ]:
message = run_conversation("현재 위치는 어디인가요?")

{
  "id": "chatcmpl-97fscJuPdhtul67Bunt4i2D0JPEjw",
  "object": "chat.completion",
  "created": 1711616310,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_location_from_gps",
          "arguments": "{}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 108,
    "completion_tokens": 9,
    "total_tokens": 117
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}
## messages ##
[{'role': 'user', 'content': '현재 위치는 어디인가요?'}, <OpenAIObject at 0x7d1d92fa75b0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}, {'role': 'function', 'name': 'get_location_from_gps', 'content': '서

In [ ]:
message.choices[0].message.content

'저는 현재 서울에 위치해 있습니다.'

In [ ]:
message = run_conversation("서울의 날씨는요?")

{
  "id": "chatcmpl-97fsd0aW33Pq8ORSIpky0mryRDuzv",
  "object": "chat.completion",
  "created": 1711616311,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 109,
    "completion_tokens": 17,
    "total_tokens": 126
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': '서울의 날씨는요?'}, <OpenAIObject at 0x7d1d92fa4e50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\

In [ ]:
message.choices[0].message.content

'현재 서울의 온도는 72°F이며, 날씨는 화창하고 바람이 강합니다.'

In [ ]:
response = run_conversation("안녕하세요.")

{
  "id": "chatcmpl-97fseSja9WSHxhWg2nw5oJIJUCfLp",
  "object": "chat.completion",
  "created": 1711616312,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc548\ub155\ud558\uc138\uc694! \uc5b4\ub5bb\uac8c \ub3c4\uc640\ub4dc\ub9b4\uae4c\uc694?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 103,
    "completion_tokens": 23,
    "total_tokens": 126
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": "\uc548\ub155\ud558\uc138\uc694! \uc5b4\ub5bb\uac8c \ub3c4\uc640\ub4dc\ub9b4\uae4c\uc694?"
}


In [ ]:
response.content

'안녕하세요! 어떻게 도와드릴까요?'

In [ ]:
response = run_conversation("현재 날씨는?")

{
  "id": "chatcmpl-97fsfBEGOeluwmizc038a8kwmEaZA",
  "object": "chat.completion",
  "created": 1711616313,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_location_from_gps",
          "arguments": "{}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 106,
    "completion_tokens": 9,
    "total_tokens": 115
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}
## messages ##
[{'role': 'user', 'content': '현재 날씨는?'}, <OpenAIObject at 0x7d1d92fa52b0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}, {'role': 'function', 'name': 'get_location_from_gps', 'content': '서울'}]
#